# Layer Specific DMR
- Run DMR on each region separately, compare layers
- Combine layer hypo DMR from all regions, reannotate and scan the concatenated bed file

In [2]:
import pathlib
import pandas as pd
import pybedtools
from papermill import execute_notebook

In [3]:
dmr_dir = pathlib.Path('LayerSpecificDMR/')
layers = ['IT-L23', 'IT-L4', 'IT-L5', 'IT-L6']
output_dir = pathlib.Path('AggLayerDMR')
output_dir.mkdir(exist_ok=True)

## Save total bed

In [3]:
for layer in layers:
    this_output_dir = output_dir / layer
    this_output_dir.mkdir(exist_ok=True)
    bed_list = list(dmr_dir.glob(f'*/HypoDMR/*{layer}*bed'))
    total_bed_path = this_output_dir / f'{layer}.total.bed'
    with open(total_bed_path, 'w') as f:
        for path in bed_list:
            with open(path) as _f:
                f.writelines(_f.readlines())
    total_bed = pybedtools.BedTool(str(total_bed_path)).sort().merge()
    total_bed.saveas(str(total_bed_path))
    
    # add index
    total_bed = pd.read_csv(total_bed_path, header=None, sep='\t')
    total_bed.index = layer + '_' + total_bed.index.astype(str)

    total_bed.reset_index().iloc[:, [1, 2, 3, 0]].to_csv(total_bed_path, sep='\t', index=None, header=None)

## Motif Scan

In [4]:
motif_file_path='/home/hanliu/ref/MEME/selected_motifs_sets/JASPAR2020_CORE_vertebrates_non-redundant_pfms.has_mouse_id.meme'
genome_fasta_path='/home/hanliu/ref/mouse/genome/fasta/raw/mm10.fa'
chrom_size_path='/home/hanliu/ref/mouse/genome/mm10.main.chrom.sizes'

In [5]:
for layer in layers:
    bed_path = f'/home/hanliu/project/mouse_rostral_brain/study/ITSpatial/DMRAnalysis/AggLayerDMR/{layer}/{layer}.total.bed'
    params = dict(bed_path=bed_path,
                  motif_file_path=motif_file_path,
                  genome_fasta_path=genome_fasta_path,
                  chrom_size_path=chrom_size_path)

    input_path = '/home/hanliu/project/mouse_rostral_brain/DMR/DMRRecipe/2.MotifScan.ipynb'
    output_path = pathlib.Path(bed_path).parent / '2.MotifScan.ipynb'
    execute_notebook(str(input_path),
                     str(output_path),
                     parameters=params,
                     engine_name=None,
                     prepare_only=False,
                     kernel_name=None,
                     progress_bar=True,
                     log_output=False,
                     start_timeout=60,
                     report_mode=False,
                     cwd=str(output_dir))
    pass

## Genome annot

In [5]:
for layer in layers:
    bed_path = f'/home/hanliu/project/mouse_rostral_brain/study/ITSpatial/DMRAnalysis/AggLayerDMR/{layer}/{layer}.total.bed'
    params = dict(dmr_bed=bed_path)
    input_path = '/home/hanliu/project/mouse_rostral_brain/DMR/DMRRecipe/3.DMRAnnotation.ipynb'
    output_path = pathlib.Path(bed_path).parent / '3.DMRAnnotation.ipynb'
    execute_notebook(str(input_path),
                     str(output_path),
                     parameters=params,
                     engine_name=None,
                     prepare_only=False,
                     kernel_name=None,
                     progress_bar=True,
                     log_output=False,
                     start_timeout=60,
                     report_mode=False,
                     cwd=str(output_dir))